In [8]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from geopy.distance import geodesic
from shapely.ops import unary_union

In [12]:
# 파일 로드
df_welfare = pd.read_csv('../welfare_school_concat_EPSG_4326.csv')

In [13]:
df_cctv = pd.read_csv("./서울시_cctv_데이터.csv", header=None)

# 열 이름 추가
column_names = ['번호', '자치구명', '안심주소', 'CCTV용도', '위도', '경도', 'CCTV수량', '수정일시', '경찰서명', '경찰서부서명', '경찰서부서코드', '시도경찰청명']
df_cctv.columns = column_names

# '번호' 열을 새로운 열 인덱스로 설정
df_cctv.set_index('번호', inplace=True)

# 결과 확인
df_cctv

자치구명                                 안심주소 CCTV용도         위도          경도  \
번호                                                                              
1      서초구                           양재2동 산4-14   산불감시  37.432124  127.047291   
2      금천구          시흥대로2길 8-49 주변 [시흥3동 978-6]     방범  37.434221  126.903997   
3      금천구          시흥대로2길 8-49 주변 [시흥3동 978-6]     방범  37.434303  126.903990   
4      금천구          시흥대로2길 8-49 주변 [시흥3동 978-6]     방범  37.434312  126.904080   
5      금천구          시흥대로2길 8-49 주변 [시흥3동 978-6]     방범  37.434383  126.903963   
...    ...                                  ...    ...        ...         ...   
61515  도봉구                      도봉동 8번지 다락원체육공원     방범  37.692401  127.046621   
61516  도봉구              창포원로 45(도봉2동 8다락원 체육공원)     방범  37.692401  127.046621   
61517  도봉구              창포원로 45(도봉2동 8다락원 체육공원)     방범  37.692464  127.047108   
61518  도봉구              창포원로 45(도봉2동 8다락원 체육공원)     방범  37.692900  127.047000   
61519  도봉구  평화로15번길 9-23 주변(도봉1동 380 서울인강학교 방향)     방범  37.693879  127.042846   

       CCTV수량        수정일시     경찰서명 경찰서부서명          경찰서부서코드 시도경찰청명  
번호                                                                 
1           1  2021-01-19  서울서초경찰서   서울서초  G30000001329813    서울청  
2           1  2022-02-23  서울금천경찰서   서울금천  G30000001332675    서울청  
3           1  2022-02-23  서울금천경찰서   서울금천  G30000001332675    서울청  
4           1  2022-02-23  서울금천경찰서   서울금천  G30000001332675    서울청  
5           1  2022-02-23  서울금천경찰서   서울금천  G30000001332675    서울청  
...       ...         ...      ...    ...              ...    ...  
61515       2  2022-05-17  서울도봉경찰서   서울도봉  G30000001329939    서울청  
61516       1  2022-05-17  서울도봉경찰서   서울도봉  G30000001329939    서울청  
61517       1  2022-05-17  서울도봉경찰서   서울도봉  G30000001329939    서울청  
61518       1  2022-05-17  서울도봉경찰서   서울도봉  G30000001329939    서울청  
61519       1  2022-05-17  서울도봉경찰서   서울도봉  G30000001329939    서울청  

[61519 rows x 11 columns]

In [14]:
# GeoDataFrame 생성
gdf_welfare = gpd.GeoDataFrame(df_welfare, geometry=gpd.points_from_xy(df_welfare['X'], df_welfare['Y']), crs='epsg:4326')
gdf_cctv = gpd.GeoDataFrame(df_cctv, geometry=gpd.points_from_xy(df_cctv['경도'], df_cctv['위도']), crs='epsg:4326')

# 300m 반경 내의 행 개수 계산
result = []

for index, row in gdf_welfare.iterrows():
    buffer = row['geometry'].buffer(0.003)  # 0.003은 300m를 의미합니다. (단위는 좌표계에 따라 다름)
    within_cctv = gdf_cctv[gdf_cctv.within(buffer)]
    result.append({'시설명': row['시설명'], 'cctv 개수': len(within_cctv)})

cctv_count = pd.DataFrame(result)
cctv_count

시설명  cctv 개수
0        송파구방이복지관       31
1      서울시각장애인복지관       18
2    서울시립발달장애인복지관        0
3           청음복지관      100
4      마포장애인종합복지관       56
..            ...      ...
675        서울정문학교       73
676        서울정진학교       19
677        한국우진학교       24
678         서울농학교       46
679         서울맹학교       55

[680 rows x 2 columns]